In [ ]:
import numpy as np

# Define system parameters
m = 1.0     # Mass
c = 0.1     # Damping
k = 1.0     # Stiffness

# Define Newmark scheme parameters
beta = 0.25
gamma = 0.5

# Define time stepping parameters
t_end = 10.0
dt = 0.1
n_steps = int(t_end/dt)

# Initialize arrays
t = np.linspace(0, t_end, n_steps)
u = np.zeros(n_steps)
v = np.zeros(n_steps)
a = np.zeros(n_steps)

# Initial conditions
u[0] = 0.0
v[0] = 0.0
a[0] = 0.0

# Time stepping
for i in range(1, n_steps):
    # Calculate effective mass, damping and force
    m_eff = m + gamma*c*dt + beta*k*dt*dt
    c_eff = c + beta*k*dt
    f_eff = m*a[i-1] + c*v[i-1] + k*u[i-1]

    # Calculate acceleration, velocity and displacement
    a[i] = (f_eff - c_eff*v[i-1] - k*u[i-1]) / m_eff
    v[i] = v[i-1] + dt*((1-gamma)*a[i-1] + gamma*a[i])
    u[i] = u[i-1] + dt*v[i-1] + dt*dt*((0.5-beta)*a[i-1] + beta*a[i])

In [ ]:
import numpy as np

# Define system parameters
m = 1.0     # Mass
c = 0.1     # Damping
k = 1.0     # Stiffness

# Define Wilson Theta method parameter
theta = 1.4

# Define time stepping parameters
t_end = 10.0
dt = 0.1
n_steps = int(t_end/dt)

# Initialize arrays
t = np.linspace(0, t_end, n_steps)
u = np.zeros(n_steps)
v = np.zeros(n_steps)
a = np.zeros(n_steps)

# Initial conditions
u[0] = 0.0
v[0] = 0.0
a[0] = 0.0

# Time stepping
for i in range(1, n_steps):
    # Calculate effective mass, damping and force
    m_eff = 6.0*m/(theta*dt)**2
    c_eff = 3.0*c/(theta*dt)
    k_eff = k - 2.0*c_eff/(3.0*dt) - m_eff/(3.0*dt**2)
    
    # Calculate predictor values
    u_p = u[i-1] + dt*v[i-1] + dt**2/2.0*(1-2*theta)*a[i-1]
    v_p = v[i-1] + (1-theta)*dt*a[i-1]
    a_p = (v_p - v[i-1])/((1-theta)*dt) - (1-2*theta)*a[i-1]
    
    # Calculate corrector values
    u[i] = u_p + (theta*dt)**2/6.0*(a_p - (1-2*theta)*a[i-1])
    v[i] = v_p + theta*dt/2.0*(a_p - a[i-1])
    a[i] = a_p

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# parameters
m = 1.0  # mass
k = 1.0  # stiffness
f = 1.0  # external force

# time stepping parameters
t_end = 10.0
dt = 0.01
n_steps = int(t_end/dt)

# Initialize arrays
t = np.linspace(0, t_end, n_steps)
u = np.zeros(n_steps)  # displacement
v = np.zeros(n_steps)  # velocity

# Initial conditions
u[0] = 0
v[0] = 0

# Finite Element time stepping
for i in range(1, n_steps):
    # Predictor
    u[i] = u[i-1] + dt * v[i-1]
    v[i] = v[i-1] + dt * (f - k * u[i-1]) / m

# Plot the solution
plt.figure()
plt.plot(t, u, label='Displacement')
plt.plot(t, v, label='Velocity')
plt.legend()
plt.show()

In [ ]:
import numpy as np

class FEState:
    def __init__(self, mass_matrix, damping, stiffness_matrix, internal_forces):
        self.mass_matrix = mass_matrix
        self.damping = damping
        self.stiffness_matrix = stiffness_matrix
        self.internal_forces = internal_forces
        self.displacement = np.zeros_like(internal_forces)
        self.velocity = np.zeros_like(internal_forces)
        self.acceleration = np.zeros_like(internal_forces)

    def compute_internal_forces(self):
        return self.stiffness_matrix @ self.displacement - self.internal_forces

    def update_displacement(self, dt):
        self.displacement += dt * self.velocity + 0.5 * dt**2 * self.acceleration

    def update_velocity(self, dt):
        self.velocity += dt * self.acceleration

    def update_acceleration(self):
        self.acceleration = np.linalg.solve(self.mass_matrix, self.internal_forces - self.damping @ self.velocity - self.stiffness_matrix @ self.displacement)

def explicit_time_stepping(festate, dt, t_final):
    t = 0.0
    while t < t_final:
        festate.update_displacement(dt)
        festate.update_velocity(dt)
        festate.update_acceleration()
        t += dt

Could generate a script implementing an explicit algorithm for time stepping using of a single material point with a mass m, which is connected to a rigid support via a spring. The nonlinear spring exhibits viscoplastic behavior that is characterized by the parameters E, strength f and viscosity parameter eta. There is no hardening assumed. The simulation should deliver an objective response for the specified material parameters and a loading history prescribed by the monotonically increasing displacement which enters the inelastic regime crossing 50% beyond the elastic threshold. The algorithm should return the same response for two specified values of time step specified as an algorithmic parameter.

Certainly! Here's a script that implements an explicit algorithm for time stepping using a single material point with the given characteristics:

In [ ]:
import numpy as np

def simulate(m, E, f, eta, elastic_threshold, displacement, dt_1, dt_2):
    # Material properties
    A = np.sqrt(2 * f * E)
    B = eta * np.sqrt(2 * f * E)

    # Time steps
    dt = min(dt_1, dt_2)

    # Initial state
    u = 0.0  # displacement
    v = 0.0  # velocity
    t = 0.0  # time

    # Initial elastic response
    elastic_response = []

    while u < elastic_threshold:
        # Elastic regime: Hook's law
        f_elastic = E * u

        # Update state variables
        a = (1.0 / m) * (displacement - f_elastic - B * v)
        v = v + a * dt
        u = u + v * dt

        # Collect elastic response
        elastic_response.append((t, u))

        # Increment time
        t += dt

    # Set a common state for both time steps
    u_common = u
    v_common = v
    t_common = t

    # Reset state for dt_1
    u = u_common
    v = v_common
    t = t_common

    # Reset state for dt_2
    u = u_common
    v = v_common
    t = t_common

    # Inelastic regime: viscoplastic response
    inelastic_response_dt_1 = []
    inelastic_response_dt_2 = []

    while u < displacement:
        # Viscoplastic regime: Rate dependent behavior
        f_inelastic = A * np.sign(displacement - u)

        # Update state variables for dt_1
        a_dt_1 = (1.0 / m) * (displacement - f_inelastic - B * v)
        v_dt_1 = v + a_dt_1 * dt_1
        u_dt_1 = u + v_dt_1 * dt_1
        inelastic_response_dt_1.append((t, u_dt_1))

        # Update state variables for dt_2
        a_dt_2 = (1.0 / m) * (displacement - f_inelastic - B * v)
        v_dt_2 = v + a_dt_2 * dt_2
        u_dt_2 = u + v_dt_2 * dt_2
        inelastic_response_dt_2.append((t, u_dt_2))

        # Increment time
        t += dt_1  # Assuming dt_1 and dt_2 are the same for simplicity

    return elastic_response, inelastic_response_dt_1, inelastic_response_dt_2

# Example usage
m = 1.0
E = 100.0
f = 10.0
eta = 0.2
elastic_threshold = 0.5
displacement = 1.0
dt_1 = 0.01
dt_2 = 0.001

elastic_response, inelastic_response_dt_1, inelastic_response_dt_2 = simulate(m, E, f, eta, elastic_threshold, displacement, dt_1, dt_2)

print("Elastic Response:")
for t, u in elastic_response:
    print(f"Time: {t}\tDisplacement: {u}")

print("Inelastic Response (dt_1):")
for t, u in inelastic_response_dt_1:
    print(f"Time: {t}\tDisplacement: {u}")

print("Inelastic Response (dt_2):")
for t, u in inelastic_response_dt_2:
    print(f"Time: {t}\tDisplacement: {u}")


Please note that this script assumes the given parameters and loading history as inputs and uses a simple explicit time integration scheme for the numerical simulation. It provides the elastic and inelastic responses for two specified values of time steps, dt_1 and dt_2. Make sure to adjust the parameters and loading history as per your requirements.

In [ ]:
2.7 / 0.6, 3 / 0.6

In [ ]:
1 / 0.6